In [2]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 50)

In [3]:
reg_data = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
reg_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [4]:
ncaa_data = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
ncaa_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19


In [5]:
#removing WLoc
reg_data['WLoc'].value_counts()

H    63049
A    33019
N    10766
Name: WLoc, dtype: int64

In [6]:
ncaa_data['WLoc'].value_counts()
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

N    1248
Name: WLoc, dtype: int64

In [7]:
# thinking of removing NumOT
print("Regular Season\n",reg_data['NumOT'].value_counts())
print("NCAA\n",ncaa_data['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    100790
1      5004
2       844
3       157
4        33
5         5
6         1
Name: NumOT, dtype: int64
NCAA
 0    1170
1      65
2      13
Name: NumOT, dtype: int64


In [8]:
drop_cols = ['WLoc']
reg_data = reg_data.drop(columns = drop_cols)
cols = reg_data.columns.values
print(cols)
reg_data.head()

['Season' 'DayNum' 'WTeamID' 'WScore' 'LTeamID' 'LScore' 'NumOT' 'WFGM'
 'WFGA' 'WFGM3' 'WFGA3' 'WFTM' 'WFTA' 'WOR' 'WDR' 'WAst' 'WTO' 'WStl'
 'WBlk' 'WPF' 'LFGM' 'LFGA' 'LFGM3' 'LFGA3' 'LFTM' 'LFTA' 'LOR' 'LDR'
 'LAst' 'LTO' 'LStl' 'LBlk' 'LPF']


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [10]:
# converting it to team A and team B from WTeam and LTeam
list_reg_data = []

for row in tqdm(reg_data.itertuples()):
    new_reg_data = dict()
    coin = random.random()
    new_reg_data['Season'] = row.Season
    new_reg_data['DayNum'] = row.DayNum
    new_reg_data['NumOT'] = row.NumOT

    if coin>0.5:
        new_reg_data['A'] = row.WTeamID
        try:
            new_reg_data['FGP_A'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        new_reg_data['OR_A'] = row.WOR
        new_reg_data['DR_A'] = row.WDR
        new_reg_data['AST_A'] = row.WAst
        new_reg_data['TO_A'] = row.WTO
        new_reg_data['STL_A'] = row.WStl
        new_reg_data['BLK_A'] = row.WBlk
        new_reg_data['PF_A'] = row.WPF
        new_reg_data['B'] = row.LTeamID
        try:
            new_reg_data['FGP_B'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        new_reg_data['OR_B'] = row.LOR
        new_reg_data['DR_B'] = row.LDR
        new_reg_data['AST_B'] = row.LAst
        new_reg_data['TO_B'] = row.LTO
        new_reg_data['STL_B'] = row.LStl
        new_reg_data['BLK_B'] = row.LBlk
        new_reg_data['PF_B'] = row.LPF
        new_reg_data['Win'] = 1
    else:
        new_reg_data['A'] = row.LTeamID
        try:
            new_reg_data['FGP_A'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        new_reg_data['OR_A'] = row.LOR
        new_reg_data['DR_A'] = row.LDR
        new_reg_data['AST_A'] = row.LAst
        new_reg_data['TO_A'] = row.LTO
        new_reg_data['STL_A'] = row.LStl
        new_reg_data['BLK_A'] = row.LBlk
        new_reg_data['PF_A'] = row.LPF
        new_reg_data['B'] = row.WTeamID
        try:
            new_reg_data['FGP_B'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        new_reg_data['OR_B'] = row.WOR
        new_reg_data['DR_B'] = row.WDR
        new_reg_data['AST_B'] = row.WAst
        new_reg_data['TO_B'] = row.WTO
        new_reg_data['STL_B'] = row.WStl
        new_reg_data['BLK_B'] = row.WBlk
        new_reg_data['PF_B'] = row.WPF
        new_reg_data['Win'] = 0

    list_reg_data.append(new_reg_data)

new_reg_data = pd.DataFrame(list_reg_data)
new_reg_data


106834it [00:00, 237929.36it/s]


,Season,DayNum,NumOT,A,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,B,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,Win
0,2003,10,0,1104,46.551724,21.428571,61.111111,14,24,13,23,7,1,22,1328,41.509434,20.000000,72.727273,10,22,8,18,9,2,20,1
1,2003,10,0,1272,41.935484,40.000000,52.631579,15,28,16,13,4,4,18,1393,35.820896,25.000000,45.000000,20,25,7,12,8,6,16,1
2,2003,11,0,1266,41.379310,44.444444,58.620690,17,26,15,10,5,2,25,1437,30.136986,11.538462,60.869565,31,22,9,12,2,5,23,1
3,2003,11,0,1296,47.368421,33.333333,54.838710,6,19,11,12,14,2,18,1457,36.734694,27.272727,53.333333,17,20,9,19,4,3,23,1
4,2003,11,0,1208,38.709677,37.500000,62.962963,21,15,12,10,7,1,14,1400,49.180328,42.857143,84.615385,17,22,12,14,4,4,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106829,2023,113,0,1328,35.593220,27.272727,80.000000,6,13,13,8,10,1,12,1403,61.702128,58.333333,64.285714,5,31,11,16,4,1,17,0
106830,2023,113,0,1103,44.067797,19.047619,50.000000,9,23,10,10,3,0,11,1405,52.459016,50.000000,76.923077,7,25,16,7,7,7,17,0
106831,2023,113,0,1429,41.176471,25.925926,72.727273,7,27,11,12,7,3,15,1461,31.481481,26.086957,75.000000,7,25,6,11,6,3,19,1
106832,2023,113,0,1433,52.542373,57.894737,83.333333,8,23,14,9,6,5,16,1386,37.500000,28.571429,72.222222,5,20,10,11,3,3,15,1


In [11]:
new_reg_data['Win'].value_counts()

1    53418
0    53416
Name: Win, dtype: int64

In [12]:
new_reg_data.corr()

,Season,DayNum,NumOT,A,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,B,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,Win
Season,1.000000,-0.048352,-0.007047,0.010469,0.007263,-0.024143,0.068033,-0.228328,0.094980,-0.035121,-0.193201,-0.092506,-0.005447,-0.196750,0.009190,0.009012,-0.019625,0.067523,-0.229923,0.097383,-0.032823,-0.196759,-0.087202,0.001181,-0.198157,-0.005458
DayNum,-0.048352,1.000000,0.029183,0.000554,0.023512,0.020061,0.050675,-0.053429,-0.038109,-0.034188,-0.141334,-0.060295,-0.029482,-0.036246,0.001682,0.021740,0.024473,0.052416,-0.051268,-0.039686,-0.029830,-0.143491,-0.062213,-0.027551,-0.038199,-0.000830
NumOT,-0.007047,0.029183,1.000000,-0.003484,-0.012833,-0.002454,0.005939,0.084868,0.148380,0.055521,0.061253,0.038639,0.047150,0.114258,0.003264,-0.013284,-0.006831,0.002061,0.091440,0.149069,0.048876,0.061454,0.040501,0.044867,0.118187,0.002385
A,0.010469,0.000554,-0.003484,1.000000,0.016957,0.005996,0.009204,0.008243,0.018510,0.014548,-0.022782,-0.012619,0.009348,-0.017606,0.003008,-0.018480,-0.005957,0.000245,-0.016515,-0.020037,-0.016692,-0.014777,-0.015198,-0.001672,0.005445,0.027791
FGP_A,0.007263,0.023512,-0.012833,0.016957,1.000000,0.535127,0.065437,-0.320566,0.148594,0.552272,0.000316,0.033907,0.092893,-0.069509,-0.016195,-0.091075,-0.022244,0.011166,-0.004648,-0.644805,-0.096442,-0.026239,-0.022247,-0.339567,0.053633,0.483619
FGP3_A,-0.024143,0.020061,-0.002454,0.005996,0.535127,1.000000,0.055168,-0.209696,0.051403,0.419413,-0.011490,-0.031078,0.026439,-0.031735,-0.002706,-0.024527,-0.002114,0.013094,0.024183,-0.354399,-0.031720,-0.058074,-0.024131,-0.094246,-0.004586,0.314233
FTP_A,0.068033,0.050675,0.005939,0.009204,0.065437,0.055168,1.000000,-0.119468,0.026704,0.030660,-0.056202,-0.027245,-0.006360,-0.030080,0.005319,0.001390,0.003875,0.033035,-0.027944,-0.221128,-0.014987,-0.045684,-0.043016,-0.038921,0.017632,0.142750
OR_A,-0.228328,-0.053429,0.084868,0.008243,-0.320566,-0.209696,-0.119468,1.000000,0.042805,-0.003278,0.091350,0.108418,0.081904,0.138419,-0.016094,-0.006429,0.017884,-0.034391,0.084933,0.011764,0.017355,0.134961,0.062277,0.267691,0.161307,-0.015752
DR_A,0.094980,-0.038109,0.148380,0.018510,0.148594,0.051403,0.026704,0.042805,1.000000,0.205210,0.116660,-0.118429,0.268672,-0.027789,-0.008242,-0.643880,-0.350344,-0.220523,0.006621,0.032446,-0.249029,-0.204361,0.072365,-0.046738,0.142092,0.396058
AST_A,-0.035121,-0.034188,0.055521,0.014548,0.552272,0.419413,0.030660,-0.003278,0.205210,1.000000,-0.071688,0.131189,0.136416,-0.068204,-0.013649,-0.098570,-0.031404,-0.014030,0.024438,-0.245917,0.023560,0.094333,-0.022054,-0.137493,-0.017196,0.372153


In [59]:
new_reg_data[new_reg_data['Season']==2023]

,Season,DayNum,NumOT,A,FGM_A,FGA_A,FGM3_A,FGA3_A,FTM_A,FTA_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,B,FGM_B,FGA_B,FGM3_B,FGA3_B,FTM_B,FTA_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,Win
102032,2023,7,0,1101,23,57,8,28,11,20,4,28,12,15,9,5,17,1238,22,55,2,19,10,14,7,33,9,21,6,1,21,1
102033,2023,7,1,1355,31,59,4,15,14,17,5,28,11,12,3,4,15,1103,30,69,11,31,10,12,8,23,16,9,5,5,17,0
102034,2023,7,0,1104,27,69,3,28,18,26,19,43,10,19,6,9,18,1255,19,71,5,13,11,19,12,23,8,9,8,3,22,1
102035,2023,7,0,1112,38,53,11,18,30,40,7,36,30,24,7,5,22,1311,29,72,7,18,10,22,8,14,11,16,17,1,29,1
102036,2023,7,0,1113,21,62,6,24,14,29,14,31,10,21,8,11,27,1470,19,55,2,14,19,32,5,28,10,17,12,4,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106829,2023,113,0,1403,29,47,7,12,9,14,5,31,11,16,4,1,17,1328,21,59,9,33,12,15,6,13,13,8,10,1,12,1
106830,2023,113,0,1405,32,61,10,20,10,13,7,25,16,7,7,7,17,1103,26,59,4,21,7,14,9,23,10,10,3,0,11,1
106831,2023,113,0,1461,17,54,6,23,15,20,7,25,6,11,6,3,19,1429,21,51,7,27,16,22,7,27,11,12,7,3,15,0
106832,2023,113,0,1386,21,56,8,28,13,18,5,20,10,11,3,3,15,1433,31,59,11,19,15,18,8,23,14,9,6,5,16,0
